In [1]:
import os
import json
import random
from collections import defaultdict

# ============================================
# PARAMETRELER - BURADAN AYARLAYIN
# ============================================
CEFR_LEVEL = "B1"  # "A1", "A2", "B1", "B2", "C1"
GROUP_NUMBER = 1    # 1, 2, 3, ... (kaçıncı grup olduğu)

print(f"🎯 Seviye: {CEFR_LEVEL}")
print(f"📊 Grup: {GROUP_NUMBER}")
print("="*60)

# Dizinleri ayarla
current_dir = os.getcwd()
data_dir = os.path.join(current_dir, "..", "data", "generated_sentences", CEFR_LEVEL)
data_dir = os.path.abspath(data_dir)

output_base_dir = os.path.join(current_dir, "..", "data", "tasks")
os.makedirs(output_base_dir, exist_ok=True)

# Model klasörlerini bul
model_dirs = [d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))]
model_dirs.sort()

print(f"📁 Bulunan modeller: {model_dirs}")
print(f"📊 Toplam {len(model_dirs)} model bulundu")

# Her modelden belirtilen grup numarasındaki dosyayı oku
grouped_data = defaultdict(list)

for model_dir in model_dirs:
    model_path = os.path.join(data_dir, model_dir)
    
    # Belirtilen grup numarasındaki dosyayı seç
    # Format: ModelName_Level_GroupNumber.json
    target_file = f"{model_dir}_{CEFR_LEVEL}_{GROUP_NUMBER}.json"
    file_path = os.path.join(model_path, target_file)
    
    if not os.path.exists(file_path):
        print(f"⚠️  Dosya bulunamadı: {target_file}")
        continue
    
    # JSON dosyasını oku
    with open(file_path, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    model_name = data["model_name"]
    words = data["words"]
    sentences = data["sentences"]
    
    print(f"✅ {model_name}: {len(words)} kelime")
    
    # Her kelime için veriyi grupla
    for word, sentence in zip(words, sentences):
        grouped_data[word].append({
            "model": model_name,
            "sentence": sentence
        })

print(f"\n📝 Toplam {len(grouped_data)} benzersiz kelime bulundu")
print("="*60)

# Task'ları oluştur
tasks = []

# Dinamik label oluştur (model sayısına göre)
num_models = len(model_dirs)
labels = [f"Sentence {chr(65+i)}" for i in range(num_models)]  # A, B, C, D, E, F, ...
print(f"🏷️  Label'lar: {labels}")

# Prompt şablonu - dinamik olarak model sayısını kullan
sentences_output_format = "\n".join([f"{label}: <Word Usage>, <Level Appropriateness>, <Grammar>, <Naturalness>" for label in labels])

prompt_template = """You are a professional CEFR-aligned English sentence evaluator.

Your task is to evaluate {num_sentences} example sentences that all use the target word: "{{word}}" at CEFR level: {{level}}.

Rate each sentence from 1 (poor) to 5 (excellent) for the following **four independent criteria**:

1) Word Usage — Is the given word used with the correct meaning and appropriately in context? **If the word is not used at all, give the lowest score.**
2) Level Appropriateness — Are the tense, structure, and syntax appropriate for the target CEFR level (A1, A2, B1, B2, C1)?
3) Grammatical Accuracy — Are the grammatical structures correct and suitable for the expected level (simple / intermediate / advanced)?
4) Naturalness — Does the sentence sound natural and align with standard usage by native English speakers?

⚠️ Important Instructions:
- **Only return numerical ratings** for each criterion.
- **Do not include any explanations, comments, or justifications.**
- Follow the exact output format below.

### Output Format:
{output_format}

### Sentences:
{{sentences_block}}
"""

prompt_template = prompt_template.format(
    num_sentences=num_models,
    output_format=sentences_output_format
)

for word, sentence_data in grouped_data.items():
    if len(sentence_data) != num_models:
        print(f"⚠️  '{word}' için {len(sentence_data)} cümle var ({num_models} gerekli), atlanıyor.")
        continue
    
    # Cümleleri karıştır
    random.shuffle(sentence_data)
    
    labeled_sentences = []
    mapping = {}
    sentences_block_lines = []
    
    for label, item in zip(labels, sentence_data):
        sentence = item["sentence"]
        labeled_sentences.append((label, sentence))
        mapping[label] = {
            "model": item["model"],
            "level": CEFR_LEVEL,
            "word": word,
            "sentence": sentence
        }
        sentences_block_lines.append(f"{label}: {sentence}")
    
    # Promptu oluştur
    sentences_block = "\n".join(sentences_block_lines)
    prompt = prompt_template.replace("{{word}}", word).replace("{{level}}", CEFR_LEVEL).replace("{{sentences_block}}", sentences_block)
    
    # Task nesnesi
    task = {
        "task_id": f"{CEFR_LEVEL}_{word}_{GROUP_NUMBER}",
        "level": CEFR_LEVEL,
        "group": GROUP_NUMBER,
        "word": word,
        "labeled_sentences": labeled_sentences,
        "mapping": mapping,
        "prompt": prompt
    }
    
    tasks.append(task)

# Dosyayı kaydet - tasks klasörü direkt olarak output_base_dir altında
output_filename = f"tasks_{CEFR_LEVEL}_{GROUP_NUMBER}.json"
output_filepath = os.path.join(output_base_dir, output_filename)

with open(output_filepath, "w", encoding="utf-8") as f:
    json.dump(tasks, f, ensure_ascii=False, indent=2)

print(f"\n✅ {len(tasks)} görev oluşturuldu!")
print(f"📁 Dosya: {output_filepath}")
print("="*60)

🎯 Seviye: B1
📊 Grup: 1
📁 Bulunan modeller: ['Claude_Sonnet_4.5', 'Gemini_Pro_2.5', 'Llama-3.2-1B-Instruct', 'Llama-3.2-1B-Instruct-FineTuned', 'Llama-3.2-8B-Instruct', 'mistralai_Ministral-8B-Instruct-2410']
📊 Toplam 6 model bulundu
✅ Claude_Sonnet_4.5: 10 kelime
✅ Gemini_Pro_2.5: 10 kelime
✅ Llama-3.2-1B-Instruct: 10 kelime
✅ Llama-3.2-1B-Instruct-FineTuned: 10 kelime
✅ Llama-3.2-8B-Instruct: 10 kelime
✅ mistralai_Ministral-8B-Instruct-2410: 10 kelime

📝 Toplam 10 benzersiz kelime bulundu
🏷️  Label'lar: ['Sentence A', 'Sentence B', 'Sentence C', 'Sentence D', 'Sentence E', 'Sentence F']

✅ 10 görev oluşturuldu!
📁 Dosya: /home/muhammet/Documents/Tez/LLM_Degerlendirme/notebooks/../data/tasks/tasks_B1_1.json


In [ ]:
# 📖 Kullanım Kılavuzu

## Parametreler:
- **CEFR_LEVEL**: Hangi seviye için task oluşturulacak (A1, A2, B1, B2, C1)
- **GROUP_NUMBER**: Hangi grup/batch için (1, 2, 3, ...)

## Örnek:
```python
CEFR_LEVEL = "A1"
GROUP_NUMBER = 1
```

## Özellikler:
- ✅ **Dinamik model sayısı**: Klasörde kaç model varsa hepsini kullanır (5, 6, 7... fark etmez)
- ✅ **Otomatik label**: Model sayısına göre A, B, C, D, E, F... oluşturur
- ✅ **Yeni dosya formatı**: `ModelName_Level_GroupNumber.json`
- ✅ **Word Usage kontrolü**: Eğer kelime kullanılmadıysa en düşük puan verilmesi gerektiği prompt'a eklendi

## Çıktı:
- **Dosya**: `data/tasks/tasks_A1_1.json` (Seviye bazlı alt klasör YOK)
- Her kelime için N model cümlesi karıştırılır ve değerlendirilmek üzere hazırlanır

## Dosya Yapısı:
```
data/
  generated_sentences/
    A1/
      Llama-3.2-1B-Instruct/
        Llama-3.2-1B-Instruct_A1_1.json    ← GROUP_NUMBER = 1
        Llama-3.2-1B-Instruct_A1_2.json    ← GROUP_NUMBER = 2
      Claude_Sonnet_4.5/
        Claude_Sonnet_4.5_A1_1.json
        Claude_Sonnet_4.5_A1_2.json
      Gemini_Pro_2.5/
        Gemini_Pro_2.5_A1_1.json
        ...
  tasks/
    tasks_A1_1.json       ← Oluşturulan dosya (direkt tasks altında)
    tasks_A1_2.json
    tasks_B1_1.json
    tasks_B2_1.json
```

## ⚠️ Önemli:
Her model için **aynı grup numarasında dosya olmalı**. Örneğin Grup 1 için:
- Llama-3.2-1B-Instruct_A1_1.json ✅
- Claude_Sonnet_4.5_A1_1.json ✅
- Gemini_Pro_2.5_A1_1.json ✅

## Prompt Değişiklikleri:
- Word Usage kriterine eklendi: "If the word is not used at all, give the lowest score."